# ArXiv Semantic Graph – Project Demo

This notebook demonstrates the main steps of our pipeline:

1. Exploratory Data Analysis (EDA)
2. Embeddings (Universal Sentence Encoder)
3. HNSW index for fast nearest neighbours
4. Distance histogram and global threshold τ
5. Semantic graph construction
6. Graph clustering (Louvain communities)
7. Semantic recommendation demo


## Section 0 - Imports and paths

In [1]:
# Always reload local project modules automatically
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/home/zealot/DTU/Computational_Tools/src")

# ============================
# SECTION 0 — PROJECT SETUP
# ============================

from pathlib import Path
import sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# -----------------------------
# 0.1 — Resolve project paths
# -----------------------------
# Your repo structure:
# project/
#   ├─ src/
#   ├─ outputs/
#   ├─ notebooks/
#   └─ data/

NOTEBOOK_DIR = Path.cwd()
ROOT = NOTEBOOK_DIR.parent
SRC = ROOT / "src"

print("ROOT:", ROOT)
print("SRC:", SRC)

# -----------------------------
# 0.2 — Add src/ to PYTHONPATH
# -----------------------------
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

print("\nPYTHONPATH updated. Current sys.path first entries:")
print("\n".join(sys.path[:3]))

# -----------------------------
# 0.3 — Project imports
# -----------------------------
from arxiv_semantic_graph import eda, embeddings, graph, graph_clustering, recommend

print("\n[OK] Project modules imported successfully.")

# -----------------------------
# 0.4 — Paths to data and outputs
# -----------------------------
DATA_PATH = ROOT / "data" / "arxiv-metadata-oai-snapshot.json"

OUTPUTS = ROOT / "outputs"
OUTPUTS.mkdir(exist_ok=True)

EMB_DIR = OUTPUTS / "embeddings"
if not EMB_DIR.exists():
    print(
        f"Embeddings directory not found at:\n{EMB_DIR}\n"
        "Run the embedding extraction pipeline first."
    )

# -----------------------------
# 0.5 — Locate embedding shards
# -----------------------------
emb_paths = sorted(EMB_DIR.glob("emb_*.npy"))
if not emb_paths:
    print(f"No embedding shards found in {EMB_DIR}")

print(f"\n[Info] Found {len(emb_paths)} embedding shards.")


ROOT: /home/zealot/DTU/Computational_Tools
SRC: /home/zealot/DTU/Computational_Tools/src

PYTHONPATH updated. Current sys.path first entries:
/home/zealot/micromamba/envs/arxiv/lib/python311.zip
/home/zealot/micromamba/envs/arxiv/lib/python3.11
/home/zealot/micromamba/envs/arxiv/lib/python3.11/lib-dynload

[OK] Project modules imported successfully.
Embeddings directory not found at:
/home/zealot/DTU/Computational_Tools/outputs/embeddings
Run the embedding extraction pipeline first.
No embedding shards found in /home/zealot/DTU/Computational_Tools/outputs/embeddings

[Info] Found 0 embedding shards.


## Section 1 - EDA

In [2]:
years, lengths = eda.load_eda_data(
    file_path=str(DATA_PATH),
    max_papers=50000,
    min_year=2010,
)

stats = eda.compute_eda_stats(years, lengths)

eda.plot_year_distribution(years, save_path=None, show=True)
eda.plot_length_distribution(lengths, save_path=None, show=True)

## Section 2 -Embeddings

In [3]:
import sys, tensorflow as tf
print(sys.executable)
print(tf.__version__)
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices("GPU"))


/home/zealot/micromamba/envs/arxiv/bin/python
2.15.0
True
[]


In [4]:
DATA_PATH = Path("../data/arxiv-metadata-oai-snapshot.json")
EMB_DIR = Path("../outputs/embeddings")
EMB_DIR.mkdir(exist_ok=True)

total_embedded = embeddings.run_embeddings(
    file_path=str(DATA_PATH),
    out_dir=str(EMB_DIR),
    batch_size=64,
    shard_size=25000,
    min_year=2024,
    min_words=200,
    use_gpu=True,           # ← ENABLE GPU
    vram_mib=4096,          # ← OPTIONAL: cap GPU memory
    force_recompute=False,
)

print("Total embedded:", total_embedded)


[CPU] Running without GPU (or no GPU detected)
[Embeddings] Reading from: ../data/arxiv-metadata-oai-snapshot.json
[Embeddings] Writing shards to: ../outputs/embeddings
[Embeddings] Filters: min_year=2024, min_words=200
[Embeddings] Batch size: 64, Shard size: 25000
[USE] Loading Universal Sentence Encoder...
[USE] Model loaded successfully
[Embeddings] Starting streaming and embedding...
[Shard 0] saved 25000 papers (total 25000)
[Shard 1] saved 25000 papers (total 50000)
[Shard 2] saved 25000 papers (total 75000)
[Shard 3] saved 25000 papers (total 100000)
[Shard 4] saved 25000 papers (total 125000)
[Shard 5] saved 23477 papers (total 148477)
[Embeddings] Complete! Total papers embedded: 148,477
[Embeddings] Verification: 6 shards, 148,477 vectors, dim=512
Total embedded: 148477


## Section 3 - HNSW

In [5]:
import glob
import importlib
import arxiv_semantic_graph.graph as g

importlib.reload(g)
from arxiv_semantic_graph.graph import load_shapes, build_or_load_index


# 3.1 – Locate embedding shards
emb_paths = sorted(glob.glob(str(EMB_DIR / "emb_*.npy")))
assert emb_paths, f"No emb_*.npy files found in {EMB_DIR}"

N, D = load_shapes(emb_paths)
print(f"[Info] Total vectors: {N:,} | Dimension: {D}")

# 3.2 – Build or load HNSW index
index = build_or_load_index(
    emb_paths=emb_paths,
    dim=D,
    out_dir=OUTPUTS / "hnsw",
    efc=200,   # ef_construction (controls construction quality vs speed)
    M=16,      # maximum number of connections per node
    threads=8,
)

# 3.3 – Quick sanity check: k-NN query for a random paper
import random
import numpy as np

random_id = random.randint(0, N - 1)

# Load the actual embedding for that paper (better than using a dummy vector)
offset = 0
query_vec = None
for p in emb_paths:
    arr = np.load(p, mmap_mode="r")
    n = arr.shape[0]
    if random_id < offset + n:
        query_vec = arr[random_id - offset].astype("float32")
        break
    offset += n

assert query_vec is not None, "Could not locate embedding for random_id"

labels, dists = index.knn_query(query_vec.reshape(1, -1), k=5)

print(f"[HNSW] Index is ready.")
print(f"[HNSW] Example query for id {random_id} returns {labels.shape[1]} neighbors.")
print("Neighbor IDs:", labels[0])
print("Distances:   ", dists[0])


[Info] Total vectors: 148,477 | Dimension: 512
[HNSW] Building index…
  Added 25000/148477
  Added 50000/148477
  Added 75000/148477
  Added 100000/148477
  Added 125000/148477
  Added 148477/148477
[HNSW] Index is ready.
[HNSW] Example query for id 85714 returns 5 neighbors.
Neighbor IDs: [85714 95095 41168  7215 99717]
Distances:    [8.9406967e-07 2.2669423e-01 2.2755516e-01 2.4245679e-01 2.4412102e-01]


## Section 4 — Distance histogram & τ

In [6]:
# ---------------------------
# Section 3 — HNSW Index
# ---------------------------

from pathlib import Path
import glob
import numpy as np
from arxiv_semantic_graph.graph import load_shapes, build_or_load_index

print("[Section 3] Building or loading HNSW index...")

# 3.1 Locate embedding shards
EMB_DIR = ROOT / "outputs" / "embeddings"
emb_paths = sorted(glob.glob(str(EMB_DIR / "emb_*.npy")))
assert emb_paths, f"No embedding shards found in: {EMB_DIR}"

# 3.2 Get total vectors (N) and dimensionality (D)
N, D = load_shapes(emb_paths)
print(f"[Info] Total vectors = {N:,} | Dimension = {D}")

# 3.3 Build or load HNSW index
index = build_or_load_index(
    emb_paths=emb_paths,
    dim=D,
    out_dir=OUTPUTS / "hnsw",
    efc=200,
    M=16,
    threads=8,
)

print("[Section 3] HNSW index ready.")
print()


[Section 3] Building or loading HNSW index...
[Info] Total vectors = 148,477 | Dimension = 512
[HNSW] Loaded existing index.
[Section 3] HNSW index ready.



In [7]:
from arxiv_semantic_graph.graph import (
    compute_knn_distance_histogram,
    choose_tau_from_percentile,
)

print("[Section 4] Computing k-NN distance histogram...")

hist_result = compute_knn_distance_histogram(
    emb_dir=str(EMB_DIR),
    index=index,
    k=6,                      # 1 (self) + 5 neighbors
    out_dir=str(OUTPUTS / "eda")   # Stores plots & stats
)

hist_summary = hist_result["summary"]
hist_summary


[Section 4] Computing k-NN distance histogram...
[Histogram] Computing distance distributions with k=6…
  Processed 100000 vectors
[Histogram] Statistics: {
  "mean": 0.25393904993931454,
  "median": 0.25344739854335785,
  "std": 0.0445670277501146,
  "min": -1.1920928955078125e-07,
  "max": 0.5962079763412476
}


{'mean': 0.25393904993931454,
 'median': 0.25344739854335785,
 'std': 0.0445670277501146,
 'min': -1.1920928955078125e-07,
 'max': 0.5962079763412476}

In [8]:
import matplotlib.pyplot as plt
import numpy as np

bins = hist_result["bins"]
global_hist = hist_result["global_hist"]
centers = 0.5 * (bins[1:] + bins[:-1])

plt.figure(figsize=(10, 5))
plt.bar(centers, global_hist, width=(bins[1] - bins[0]), color="steelblue", edgecolor="black")
plt.xlabel("Cosine distance to nearest neighbors")
plt.ylabel("Frequency")
plt.title("Global k-NN Distance Distribution")
plt.grid(alpha=0.3)
plt.show()


In [9]:
# Define candidate p_keep values
pkeep_candidates = [0.05, 0.10, 0.15, 0.20, 0.25]

tau_candidates = []

for p in pkeep_candidates:
    tau = choose_tau_from_percentile(
        hist_result["bins"],
        hist_result["global_hist"],
        pkeep=p
    )
    tau_candidates.append((p, tau))

tau_candidates


[Threshold] tau ≈ 0.1900 (≈5th percentile)
[Threshold] tau ≈ 0.2100 (≈10th percentile)
[Threshold] tau ≈ 0.2200 (≈15th percentile)
[Threshold] tau ≈ 0.2200 (≈20th percentile)
[Threshold] tau ≈ 0.2300 (≈25th percentile)


[(0.05, 0.19), (0.1, 0.21), (0.15, 0.22), (0.2, 0.22), (0.25, 0.23)]

In [10]:
import pandas as pd

tau_df = pd.DataFrame(
    [{"p_keep": p, "tau": tau} for p, tau in tau_candidates]
)

tau_file = OUTPUTS / "tau_candidates.csv"
tau_df.to_csv(tau_file, index=False)

print("Saved τ candidates to:", tau_file)

tau_df


Saved τ candidates to: /home/zealot/DTU/Computational_Tools/outputs/tau_candidates.csv


,p_keep,tau
0,0.05,0.19
1,0.10,0.21
2,0.15,0.22
3,0.20,0.22
4,0.25,0.23


## Section 5 — Graph construction

In [11]:
from arxiv_semantic_graph.graph import load_shapes, build_or_load_index
# Locate embedding shards (they must already exist)
emb_paths = sorted(glob.glob(str(EMB_DIR / "emb_*.npy")))
if not emb_paths:
    raise RuntimeError(
        f"No embedding shards found in {EMB_DIR}. "
        "Run the embedding extraction pipeline before this notebook."
    )

# Basic embedding stats
N, D = load_shapes(emb_paths)
print(f"[Setup] Found {len(emb_paths)} shards: {N:,} vectors, dim={D}")


[Setup] Found 6 shards: 148,477 vectors, dim=512


In [12]:
# ==============================================================
# Section 5 — Build and evaluate graphs for multiple τ thresholds
# ==============================================================

from pathlib import Path
import pandas as pd

from arxiv_semantic_graph.graph import (
    build_graph_for_tau,
    compute_graph_stats,
)

# ----------------------------------------------
# 5.0 – Load tau candidates from the saved CSV
# ----------------------------------------------

TAU_CSV = OUTPUTS / "tau_candidates.csv"

if not TAU_CSV.exists():
    raise FileNotFoundError(
        f"Tau candidate file not found at {TAU_CSV}. "
        "Run Section 4 first to compute histogram & tau values."
    )

tau_df = pd.read_csv(TAU_CSV)
display(tau_df)

# Convert DF into list of (pkeep, tau)
tau_candidates = list(zip(tau_df["p_keep"], tau_df["tau"]))
print("\n[Info] Loaded tau candidates:")
for p, t in tau_candidates:
    print(f"  pkeep={p:.2f} → tau={t:.4f}")

index = build_or_load_index(
    emb_paths=emb_paths,
    dim=D,
    out_dir=OUTPUTS / "hnsw",
    efc=200,   # ef_construction (controls construction quality vs speed)
    M=16,      # maximum number of connections per node
    threads=8,
)

# ----------------------------------------------
# 5.1 – Prepare output directory for graph files
# ----------------------------------------------

GRAPH_DIR = OUTPUTS / "graphs"
GRAPH_DIR.mkdir(exist_ok=True)

graph_summaries = []

print("\n[Graph] Starting graph construction for each tau...\n")

# -----------------------------------------------------
# 5.2 – Build graphs for each τ + compute their metrics
# -----------------------------------------------------

seen_taus = set()
graph_summaries = []

for pkeep, tau in tau_candidates:
    tau_rounded = round(tau, 3)
    if tau_rounded in seen_taus:
        print(f"[Graph] Skipping pkeep={pkeep:.2f}: tau={tau_rounded} already used.")
        continue
    seen_taus.add(tau_rounded)

    tau_tag = f"tau_{tau_rounded:.3f}".replace(".", "p")
    out_dir = GRAPH_DIR / tau_tag
    out_dir.mkdir(parents=True, exist_ok=True)

    print("=" * 80)
    print(f"[Graph] Building graph for pkeep={pkeep:.2f}, tau≈{tau_rounded:.4f}")

    edge_path = build_graph_for_tau(
        emb_dir=str(EMB_DIR),
        index=index,
        tau=tau,
        k_for_search=50,          # Number of neighbors to retrieve
        out_dir=str(out_dir),
        sample_edges=1000,        # For inspection only
    )

    # Compute metrics
    stats = compute_graph_stats(edge_path=edge_path, num_nodes=N)

    graph_summaries.append({
        "pkeep": pkeep,
        "tau": tau,
        "nodes": stats["nodes"],
        "edges": stats["edges"],
        "avg_degree": stats["avg_degree"],
        "median_degree": stats["median_degree"],
        "isolated_nodes": stats["isolated_nodes"],
    })

# ----------------------------------------------
# 5.3 – Save results into a summary CSV
# ----------------------------------------------

graphs_df = pd.DataFrame(graph_summaries)
graphs_df_path = GRAPH_DIR / "graph_summaries.csv"
graphs_df.to_csv(graphs_df_path, index=False)

print("\n[Graph] Summary of constructed graphs:")
display(graphs_df)
print(f"[Graph] Saved summary to: {graphs_df_path}")

,p_keep,tau
0,0.05,0.19
1,0.10,0.21
2,0.15,0.22
3,0.20,0.22
4,0.25,0.23



[Info] Loaded tau candidates:
  pkeep=0.05 → tau=0.1900
  pkeep=0.10 → tau=0.2100
  pkeep=0.15 → tau=0.2200
  pkeep=0.20 → tau=0.2200
  pkeep=0.25 → tau=0.2300
[HNSW] Loaded existing index.

[Graph] Starting graph construction for each tau...

[Graph] Building graph for pkeep=0.05, tau≈0.1900
[Graph] Building edge list at tau=0.1900, k=50
  Processed nodes 0-20000
  Processed nodes 20000-25000
  Processed nodes 25000-45000
  Processed nodes 45000-50000
  Processed nodes 50000-70000
  Processed nodes 70000-75000
  Processed nodes 75000-95000
  Processed nodes 95000-100000
  Processed nodes 100000-120000
  Processed nodes 120000-125000
  Processed nodes 125000-145000
  Processed nodes 145000-148477
[Graph] Edge list written to /home/zealot/DTU/Computational_Tools/outputs/graphs/tau_0p190/edges_tau0.190.tsv
[Graph] Sample edges with titles -> /home/zealot/DTU/Computational_Tools/outputs/graphs/tau_0p190/arxiv_edge_list_sample.txt
[Stats] Computing graph statistics for /home/zealot/DTU/Co

,pkeep,tau,nodes,edges,avg_degree,median_degree,isolated_nodes
0,0.05,0.19,148477,25347,0.341427,1.0,129176
1,0.10,0.21,148477,101171,1.362783,2.0,106717
2,0.15,0.22,148477,186603,2.513561,2.0,92679
3,0.25,0.23,148477,321786,4.334490,3.0,77706


[Graph] Saved summary to: /home/zealot/DTU/Computational_Tools/outputs/graphs/graph_summaries.csv


## Section 6 - Louvain communities

In [13]:
# ============================
# SECTION 6 — Louvain clustering
# ============================

from pathlib import Path
from IPython.display import display
from arxiv_semantic_graph.graph_clustering import run_louvain

# ----------------------------------------------------
# 6.0 — Paths for graph data and Louvain output
# ----------------------------------------------------
GRAPH_DIR = OUTPUTS / "graphs"       # where edges_tau*.tsv and graph_summaries live
LOUVAIN_DIR = OUTPUTS / "louvain"    # where Louvain results will be saved
LOUVAIN_DIR.mkdir(exist_ok=True, parents=True)

print("GRAPH_DIR  :", GRAPH_DIR)
print("LOUVAIN_DIR:", LOUVAIN_DIR)


# ----------------------------------------------------
# 6.1 — Load graph summaries
# ----------------------------------------------------
graphs_path = GRAPH_DIR / "graph_summaries.csv"
if not graphs_path.exists():
    raise FileNotFoundError(f"graph_summaries.csv not found at: {graphs_path}")

graphs_df = pd.read_csv(graphs_path)
print("\n[Section 6] Loaded graph summaries:")
display(graphs_df)

# Required columns for clustering
required_cols = {"pkeep", "tau", "nodes", "edges"}
missing = required_cols - set(graphs_df.columns)
if missing:
    raise ValueError(f"Missing columns in graph_summaries.csv: {missing}")


# ----------------------------------------------------
# 6.2 — Helper: run Louvain for a single row
# ----------------------------------------------------
def run_louvain_for_row(row) -> dict:
    """
    Run Louvain clustering for a single graph configuration (one row of graphs_df).

    The file naming convention must match Section 5:
        Folder  : graphs/tau_0p190/
        Edges   : graphs/tau_0p190/edges_tau0.190.tsv

    Parameters
    ----------
    row : pandas.Series
        Contains `pkeep`, `tau`, `nodes`

    Returns
    -------
    dict with:
        pkeep, tau, modularity, num_communities, paths
    """
    pkeep = float(row["pkeep"])
    tau_raw = float(row["tau"])
    num_nodes = int(row["nodes"])

    # Round tau to 3 decimals for file naming consistency
    tau_rounded = round(tau_raw, 3)
    tau_tag = f"tau_{tau_rounded:.3f}".replace(".", "p")   # e.g. tau_0p190
    tau_str = f"{tau_rounded:.3f}"                         # e.g. 0.190

    # Construct file paths
    edge_path = GRAPH_DIR / tau_tag / f"edges_tau{tau_str}.tsv"
    out_dir = LOUVAIN_DIR / tau_tag
    out_dir.mkdir(parents=True, exist_ok=True)

    print("\n" + "=" * 80)
    print(f"[Louvain] Running for pkeep={pkeep:.3f}, tau≈{tau_raw:.6f}")
    print(f"[Louvain] Using tau={tau_str} for filenames")
    print(f"[Louvain] Edges file: {edge_path}")
    print(f"[Louvain] Output dir: {out_dir}")

    # If the edge file does not exist, skip but include a result row
    if not edge_path.exists():
        print(f"[WARNING] Edges not found, skipping: {edge_path}")
        return {
            "pkeep": pkeep,
            "tau": tau_raw,
            "modularity": None,
            "num_communities": None,
            "edges_file": str(edge_path),
            "out_dir": str(out_dir),
        }

    # Call the backend function from graph_clustering.py
    res = run_louvain(
        edge_path=str(edge_path),
        num_nodes=num_nodes,
        out_dir=str(out_dir),
    )

    return {
        "pkeep": pkeep,
        "tau": tau_raw,
        "modularity": res["modularity"],
        "num_communities": res["num_communities"],
        "edges_file": str(edge_path),
        "out_dir": str(out_dir),
    }


# ----------------------------------------------------
# 6.3 — Run Louvain for all tau values
# ----------------------------------------------------
print("\n[Section 6] Running Louvain for all graphs...")

louvain_results = [
    run_louvain_for_row(row)
    for _, row in graphs_df.iterrows()
]

louvain_df = pd.DataFrame(louvain_results)

print("\n[Louvain] Raw Louvain output:")
display(louvain_df)


# ----------------------------------------------------
# 6.4 — Merge graph metrics + Louvain metrics
# ----------------------------------------------------
quality_df = graphs_df.merge(
    louvain_df[["pkeep", "tau", "modularity", "num_communities"]],
    on=["pkeep", "tau"],
    how="left",
)

print("\n[Section 6] Combined graph quality + Louvain modularity:")
display(quality_df)

quality_path = OUTPUTS / "graph_quality_with_louvain.csv"
quality_df.to_csv(quality_path, index=False)
print(f"[Section 6] Saved summary to: {quality_path}")


# ----------------------------------------------------
# 6.5 — Select the “best” tau using modularity
# ----------------------------------------------------
valid_quality = quality_df.dropna(subset=["modularity"])

if valid_quality.empty:
    print("\n[Section 6] No valid modularity values — check graphs.")
else:
    best_row = valid_quality.sort_values("modularity", ascending=False).iloc[0]

    print("\n[Section 6] Selected τ by highest modularity:")
    print(f"  pkeep        : {best_row['pkeep']:.3f}")
    print(f"  tau          : {best_row['tau']:.6f}")
    print(f"  modularity   : {best_row['modularity']:.4f}")
    print(f"  communities  : {int(best_row['num_communities'])}")

    if "avg_degree" in best_row.index:
        print(f"  avg_degree   : {best_row['avg_degree']:.3f}")
    if "isolated_nodes" in best_row.index:
        print(f"  isolated_nodes: {int(best_row['isolated_nodes'])}")


GRAPH_DIR  : /home/zealot/DTU/Computational_Tools/outputs/graphs
LOUVAIN_DIR: /home/zealot/DTU/Computational_Tools/outputs/louvain

[Section 6] Loaded graph summaries:


,pkeep,tau,nodes,edges,avg_degree,median_degree,isolated_nodes
0,0.05,0.19,148477,25347,0.341427,1.0,129176
1,0.10,0.21,148477,101171,1.362783,2.0,106717
2,0.15,0.22,148477,186603,2.513561,2.0,92679
3,0.25,0.23,148477,321786,4.334490,3.0,77706



[Section 6] Running Louvain for all graphs...

[Louvain] Running for pkeep=0.050, tau≈0.190000
[Louvain] Using tau=0.190 for filenames
[Louvain] Edges file: /home/zealot/DTU/Computational_Tools/outputs/graphs/tau_0p190/edges_tau0.190.tsv
[Louvain] Output dir: /home/zealot/DTU/Computational_Tools/outputs/louvain/tau_0p190
[Louvain] Loading edge list from /home/zealot/DTU/Computational_Tools/outputs/graphs/tau_0p190/edges_tau0.190.tsv (N=148477)
[Louvain] Modularity = 0.9344
[Louvain] Communities found = 133000
[Louvain] Partition saved to /home/zealot/DTU/Computational_Tools/outputs/louvain/tau_0p190/louvain_partition.tsv
[Louvain] Summary saved to /home/zealot/DTU/Computational_Tools/outputs/louvain/tau_0p190/louvain_summary.json

[Louvain] Running for pkeep=0.100, tau≈0.210000
[Louvain] Using tau=0.210 for filenames
[Louvain] Edges file: /home/zealot/DTU/Computational_Tools/outputs/graphs/tau_0p210/edges_tau0.210.tsv
[Louvain] Output dir: /home/zealot/DTU/Computational_Tools/outputs/

,pkeep,tau,modularity,num_communities,edges_file,out_dir
0,0.05,0.19,0.934393,133000,/home/zealot/DTU/Computational_Tools/outputs/g...,/home/zealot/DTU/Computational_Tools/outputs/l...
1,0.10,0.21,0.868431,110964,/home/zealot/DTU/Computational_Tools/outputs/g...,/home/zealot/DTU/Computational_Tools/outputs/l...
2,0.15,0.22,0.833455,96645,/home/zealot/DTU/Computational_Tools/outputs/g...,/home/zealot/DTU/Computational_Tools/outputs/l...
3,0.25,0.23,0.804253,81233,/home/zealot/DTU/Computational_Tools/outputs/g...,/home/zealot/DTU/Computational_Tools/outputs/l...



[Section 6] Combined graph quality + Louvain modularity:


,pkeep,tau,nodes,edges,avg_degree,median_degree,isolated_nodes,modularity,num_communities
0,0.05,0.19,148477,25347,0.341427,1.0,129176,0.934393,133000
1,0.10,0.21,148477,101171,1.362783,2.0,106717,0.868431,110964
2,0.15,0.22,148477,186603,2.513561,2.0,92679,0.833455,96645
3,0.25,0.23,148477,321786,4.334490,3.0,77706,0.804253,81233


[Section 6] Saved summary to: /home/zealot/DTU/Computational_Tools/outputs/graph_quality_with_louvain.csv

[Section 6] Selected τ by highest modularity:
  pkeep        : 0.050
  tau          : 0.190000
  modularity   : 0.9344
  communities  : 133000
  avg_degree   : 0.341
  isolated_nodes: 129176


## Section 7 - Recomendation demo

In [16]:
# ============================
# SECTION 7 — Select best graph (tau) from Louvain
# ============================

import json

# 7.0 — Basic sanity checks
print("\n[Louvain] Available results:")
display(louvain_df)

# Drop rows with missing modularity or trivial graphs (0 or 1 community)
candidates = (
    louvain_df
    .dropna(subset=["modularity"])
    .query("num_communities > 1")
    .copy()
)

if candidates.empty:
    raise RuntimeError("[Louvain] No valid candidates to select from.")

# 7.1 — Select the graph with highest modularity
candidates = candidates.sort_values("modularity", ascending=False)
best_row = candidates.iloc[0]

BEST_PKEEP = float(best_row["pkeep"])
BEST_TAU = float(best_row["tau"])
BEST_MOD = float(best_row["modularity"])
BEST_COMM = int(best_row["num_communities"])

print("\n[Selection] Best graph according to modularity:")
print(f"  pkeep         : {BEST_PKEEP:.3f}")
print(f"  tau           : {BEST_TAU:.3f}")
print(f"  modularity    : {BEST_MOD:.4f}")
print(f"  communities   : {BEST_COMM}")

# 7.2 — Reconstruct folder names (must match Section 5 & 6)
tau_rounded = round(BEST_TAU, 3)
tau_tag = f"tau_{tau_rounded:.3f}".replace(".", "p")   # e.g. tau_0p190
tau_str = f"{tau_rounded:.3f}"                         # e.g. "0.190"

BEST_GRAPH_DIR = GRAPH_DIR / tau_tag
BEST_LOUVAIN_DIR = LOUVAIN_DIR / tau_tag
BEST_EDGES_PATH = BEST_GRAPH_DIR / f"edges_tau{tau_str}.tsv"

print("\n[Selection] Resolved paths:")
print(f"  Graph dir   : {BEST_GRAPH_DIR}")
print(f"  Louvain dir : {BEST_LOUVAIN_DIR}")
print(f"  Edges file  : {BEST_EDGES_PATH}")

# 7.3 — Save selection to a small JSON config for later use (e.g. in recommend.py)
best_graph_config = {
    "pkeep": BEST_PKEEP,
    "tau": tau_rounded,
    "modularity": BEST_MOD,
    "num_communities": BEST_COMM,
    "graph_dir": str(BEST_GRAPH_DIR),
    "louvain_dir": str(BEST_LOUVAIN_DIR),
    "edges_path": str(BEST_EDGES_PATH),
}

BEST_CONFIG_PATH = OUTPUTS / "best_graph_config.json"
with BEST_CONFIG_PATH.open("w", encoding="utf-8") as f:
    json.dump(best_graph_config, f, indent=2)

print(f"\n[Selection] Best graph configuration saved to: {BEST_CONFIG_PATH}")



[Louvain] Available results:


,pkeep,tau,modularity,num_communities,edges_file,out_dir
0,0.05,0.19,0.934393,133000,/home/zealot/DTU/Computational_Tools/outputs/g...,/home/zealot/DTU/Computational_Tools/outputs/l...
1,0.10,0.21,0.868431,110964,/home/zealot/DTU/Computational_Tools/outputs/g...,/home/zealot/DTU/Computational_Tools/outputs/l...
2,0.15,0.22,0.833455,96645,/home/zealot/DTU/Computational_Tools/outputs/g...,/home/zealot/DTU/Computational_Tools/outputs/l...
3,0.25,0.23,0.804253,81233,/home/zealot/DTU/Computational_Tools/outputs/g...,/home/zealot/DTU/Computational_Tools/outputs/l...



[Selection] Best graph according to modularity:
  pkeep         : 0.050
  tau           : 0.190
  modularity    : 0.9344
  communities   : 133000

[Selection] Resolved paths:
  Graph dir   : /home/zealot/DTU/Computational_Tools/outputs/graphs/tau_0p190
  Louvain dir : /home/zealot/DTU/Computational_Tools/outputs/louvain/tau_0p190
  Edges file  : /home/zealot/DTU/Computational_Tools/outputs/graphs/tau_0p190/edges_tau0.190.tsv

[Selection] Best graph configuration saved to: /home/zealot/DTU/Computational_Tools/outputs/best_graph_config.json


In [56]:
from arxiv_semantic_graph import recommend
import importlib
importlib.reload(recommend)


# Suppose:
#   EMB_DIR = OUTPUTS / "embeddings"
#   index   = your loaded hnswlib.Index

recs = recommend.recommend_random(
    emb_dir=str(EMB_DIR),
    index=index,
    k=5,
)

# Or for a specific id:
recs_id42 = recommend.recommend_for_id(
    emb_dir=str(EMB_DIR),
    index=index,
    paper_id=42,
    k=5,
)



 Random query paper
ID    : 43034
Year  : 2024
Title : usfAD Based Effective Unknown Attack Detection Focused IDS Framework

[Recommend] Loaded Louvain partition for 148,477 nodes.
[Recommendations]
- (id=1585, year=2024) ✓ significant
    SeMA: Extending and Analyzing Storyboards to Develop Secure Android Apps
    distance=0.1665

- (id=13729, year=2024) ✓ significant
    NAISR: A 3D Neural Additive Model for Interpretable Shape Representation
    distance=0.1830

- (id=53453, year=2024) ✓ significant
    Structured Jet Model for Multiwavelength Observations of the Jetted
    distance=0.1856

- (id=60103, year=None) ✓ significant
    stars with SHBoost
    distance=0.1883

- (id=6511, year=None) ✓ significant
    Accelerator Technologies
    distance=0.1904

[Recommend] Loaded Louvain partition for 148,477 nodes.
